In [1]:
##########################################################################
# Created on Sat Otc 31 21:04:45 2021                                    #
# Python for Financial Analysis and Risk Management                      #
# @author: Meng Lipeng (FRM, CFA)                                        #
##########################################################################

# 6.4.1.Forward rate measurement

Item                 | Interest  | Rate     | and      | Forward  | Value
:------:             | :----:    | :----:   | :----:   | :----:   | :----:
Tenor                | 1y        | 2y       | 3y       | 4y       | 5y
Zero Rate(Continuous)| 2.0%      | 2.2%     | 2.5%     | 2.8%     | 3.0%
FV(Initial 100)      | 102.0201  | 104.4982 | 107.7884 | 111.8513 | 116.1834

In [2]:
import numpy as np

par=100
zero_rate=np.array([0.02,0.022,0.025,0.028,0.03])
T_list=np.array([1,2,3,4,5])

def f(Rf):
    from numpy import exp
    R2,R3,R4,R5=Rf
    year2=par*exp(zero_rate[0]*T_list[0])*exp(R2*T_list[0])-par*exp(zero_rate[1]*T_list[1])
    year3=par*exp(zero_rate[1]*T_list[1])*exp(R3*T_list[0])-par*exp(zero_rate[2]*T_list[2])
    year4=par*exp(zero_rate[2]*T_list[2])*exp(R4*T_list[0])-par*exp(zero_rate[3]*T_list[3])
    year5=par*exp(zero_rate[3]*T_list[3])*exp(R5*T_list[0])-par*exp(zero_rate[-1]*T_list[-1])
    return np.array([year2,year3,year4,year5])

In [3]:
import scipy.optimize as sco

R0=[0.1,0.1,0.1,0.1]
forward_rates=sco.fsolve(func=f,x0=R0)

print('Forward rate of year2 ',round(forward_rates[0],6))
print('Forward rate of year3 ',round(forward_rates[1],6))
print('Forward rate of year4 ',round(forward_rates[2],6))
print('Forward rate of year5 ',round(forward_rates[3],6))

Forward rate of year2  0.024
Forward rate of year3  0.031
Forward rate of year4  0.037
Forward rate of year5  0.038


## Generalization

$e^{R_1 T_1} e^{R_F(T_2-T_1)}=e^{R_2 T_2}$

$\Rightarrow \
R_F=R_2+(R_2-R_1)T_1 /(T_2-T_1)
$

In [4]:
def Rf(R1,R2,T1,T2):
    '''Define a function for calculating forward rate
    R1: Zero rate of tenor T1 (Continuous compound)
    R2: Zero rate of tenor T2 (Continuous compound)
    T1: Tenor1 (year)
    T2: Tenor2 (year)'''
    forward_rate=R2+(R2-R1)*T1/(T2-T1)
    return forward_rate

In [5]:
Rf_result=Rf(R1=zero_rate[:4],R2=zero_rate[1:],T1=T_list[:4],T2=T_list[1:])

print('Forward rate of year2 ',round(Rf_result[0],6))
print('Forward rate of year3 ',round(Rf_result[1],6))
print('Forward rate of year4 ',round(Rf_result[2],6))
print('Forward rate of year5 ',round(Rf_result[3],6))

Forward rate of year2  0.024
Forward rate of year3  0.031
Forward rate of year4  0.037
Forward rate of year5  0.038


# 6.4.2.Cashflow of FRA and FRA valuation

## Cashflow of FRA

In [6]:
def Cashflow_FRA(Rk,Rm,L,T1,T2,position,when):
    '''Define a function for calculating cashflow of FRA
    Rk: Fixing rate of FRA
    Rm: Reference rate on T1 for [T1,T2]
    L:  Notional amount
    T1: Tenor
    T2: Tenor, T2>T1
    postion: 'long' or 'short'
    when: when cashflow occured. 'begin' indicates occured on T1, otherwise T2'''
    if position=='long':
        if when=='begin':
            cashflow=((Rm-Rk)*(T2-T1)*L)/(1+(T2-T1)*Rm)
        else:
            cashflow=(Rm-Rk)*(T2-T1)*L
    else:
        if when=='begin':
            cashflow=((Rk-Rm)*(T2-T1)*L)/(1+(T2-T1)*Rm)
        else:
            cashflow=(Rk-Rm)*(T2-T1)*L
    return cashflow

In [7]:
# 12X15 FRA(3months loan,1year from now)
par_FRA=1e8
R_fix=0.02
Shibor_3M=0.02756 #3M Shibor when FRA expiration(after one year)
tenor1=1
tenor2=1.25

FRA_long_end=Cashflow_FRA(Rk=R_fix,Rm=Shibor_3M,L=par_FRA,T1=tenor1,T2=tenor2,position='long',when='end')
FRA_short_end=Cashflow_FRA(Rk=R_fix,Rm=Shibor_3M,L=par_FRA,T1=tenor1,T2=tenor2,position='short',when='end')
FRA_long_begin=Cashflow_FRA(Rk=R_fix,Rm=Shibor_3M,L=par_FRA,T1=tenor1,T2=tenor2,position='long',when='begin')
FRA_short_begin=Cashflow_FRA(Rk=R_fix,Rm=Shibor_3M,L=par_FRA,T1=tenor1,T2=tenor2,position='short',when='begin')

print('Cashflow occured after 1.25yrs for long ',round(FRA_long_end,2))
print('Cashflow occured after 1.25yrs for short ',round(FRA_short_end,2))
print('Cashflow occured after 1yrs for long ',round(FRA_long_begin,2))
print('Cashflow occured after 1yrs for short ',round(FRA_short_begin,2))

Cashflow occured after 1.25yrs for long  189000.0
Cashflow occured after 1.25yrs for short  -189000.0
Cashflow occured after 1yrs for long  187706.7
Cashflow occured after 1yrs for short  -187706.7


`CFA Level2 Example:Calculating value of an FRA at maturity(i.e.,cash payment at settlement)`
* 1X4 FRA(3month loan,1month from now)
* Notional principle $1 million
* Libor_3M when FRA expiration is 6%
* Contract rate is 5.32%

In [8]:
cash_settlement=Cashflow_FRA(Rk=0.0532,Rm=0.06,L=1e6,T1=1/12,T2=4/12,position='long',when='begin')
print('Cashflow for long at settlement ',round(cash_settlement,2))

Cashflow for long at settlement  1674.88


## FRA valuation

In [9]:
def Value_FRA(Rk,Rf,R,L,T1,T2,position):
    '''Define a function for FRA valuation
    Rk: Fixing rate of FRA
    Rf: Forward Reference rate on T1 for [T1,T2]
    R: Risk free rate for tenor of T2, continuous compound
    L:  Notional amount
    T1: Tenor
    T2: Tenor, T2>T1
    postion: 'long' or 'short' '''
    if position=='long':
        value=L*(Rf-Rk)*(T2-T1)*np.exp(-R*T2)
    else:
        value=L*(Rk-Rf)*(T2-T1)*np.exp(-R*T2)
    return value

In [10]:
Shibor_6M=0.02838
Shibor_9M=0.02939
Tenor1=0.5
Tenor2=0.75

FR_Shibor=Rf(R1=Shibor_6M,R2=Shibor_9M,T1=Tenor1,T2=Tenor2)
print('Forward Reference rate on T1 for [T1,T2] is ',round(FR_Shibor,6))

Forward Reference rate on T1 for [T1,T2] is  0.03141


In [11]:
Par_FRA=2e8
R_fix=0.03
R_riskfree=0.024477

FRA_short=Value_FRA(Rk=R_fix,Rf=FR_Shibor,R=R_riskfree,L=Par_FRA,T1=Tenor1,T2=Tenor2,position='short')
FRA_long=Value_FRA(Rk=R_fix,Rf=FR_Shibor,R=R_riskfree,L=Par_FRA,T1=Tenor1,T2=Tenor2,position='long')

print('FRA value for short postion is ',round(FRA_short,2))
print('FRA value for long postion is ',round(FRA_long,2))

FRA value for short postion is  -69217.59
FRA value for long postion is  69217.59


`CFA Level2 Example:Calculating value of an FRA before settlement`
* 1X4 FRA 
* Notional principle $1 million
* T1 is 20d whereas T2 is 110d
* Libor_20d is 5.7% where as Libor_110d is 5.9%
* Contract rate is 5.32%

In [12]:
Libor_20d=0.057
Libor_110d=0.059
Tenor1=20/360
Tenor2=110/360

FR_Libor=Rf(R1=Libor_20d,R2=Libor_110d,T1=Tenor1,T2=Tenor2)
print('Forward Reference rate on T1 for [T1,T2] is ',round(FR_Libor,6))

Forward Reference rate on T1 for [T1,T2] is  0.059444


In [14]:
value_long=Value_FRA(Rk=0.0532,Rf=FR_Libor,R=Libor_110d,L=1e6,T1=Tenor1,T2=Tenor2,position='long')

print('FRA value for long postion is ',round(value_long,2))

FRA value for long postion is  1533.22
